<a href="https://colab.research.google.com/github/girGitter/ScreenTime-Analysis/blob/main/ScreenTimeAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import time
import psutil
import pickle
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QLabel
from PyQt5.QtCore import QTimer


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("App Activity Tracker")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        layout = QVBoxLayout()
        self.central_widget.setLayout(layout)

        self.label = QLabel()
        layout.addWidget(self.label)

        self.graph = PlotCanvas(self, width=5, height=4)
        layout.addWidget(self.graph)

        self.app_active_time = defaultdict(float)
        self.update_graph()

    def update_graph(self):
        self.update_active_time()
        self.graph.update_plot(self.app_active_time)
        self.label.setText("Graph updated at {}".format(time.strftime("%Y-%m-%d %H:%M:%S")))
        self.save_data()
        QTimer.singleShot(1000, self.update_graph)  # Update every second

    def load_data(self):
        try:
            with open("app_active_time.pkl", "rb") as file:
                data = pickle.load(file)
                print("Loaded data:", data)
                return data
        except FileNotFoundError:
            print("Data file not found.")
            return defaultdict(float)

    def save_data(self):
        with open("app_active_time.pkl", "wb") as file:
            pickle.dump(self.app_active_time, file)

    def update_active_time(self):
        current_time = time.time()
        for proc in psutil.process_iter(['name', 'create_time']):
            app_name = proc.info['name']
            create_time = proc.info['create_time']
            active_time = current_time - create_time
            self.app_active_time[app_name] += active_time


class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig, self.ax = plt.subplots(figsize=(width, height), dpi=dpi)
        super().__init__(fig)
        self.setParent(parent)
        self.data = {}

    def update_plot(self, app_active_time):
        self.data = app_active_time
        self.ax.clear()
        apps = list(self.data.keys())
        times = np.array(list(self.data.values())) / 60  # Convert seconds to minutes
        self.ax.barh(apps, times, color='blue')
        self.ax.set_xlabel('Active Time (minutes)')
        self.ax.set_title('App Activity')

        # Adjust y-axis limits to provide space above and below the bars
        self.ax.set_ylim(-0.5, len(apps) - 0.5)

        # Adjust figure layout to automatically adjust spacing
        self.figure.tight_layout()

        self.draw()  # Refresh canvas after updating the plot

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig, self.ax = plt.subplots(figsize=(width, height), dpi=dpi)
        super().__init__(fig)
        self.setParent(parent)
        self.data = {}

    def update_plot(self, app_active_time):
        self.data = app_active_time
        self.ax.clear()
        apps = list(self.data.keys())
        times = np.array(list(self.data.values())) / 60  # Convert seconds to minutes
        self.ax.barh(apps, times, color='blue')
        self.ax.set_xlabel('Active Time (minutes)')
        self.ax.set_title('App Activity')

        # Adjust y-axis limits to provide space above and below the bars
        self.ax.set_ylim(-0.5, len(apps) - 0.5)

        self.draw()  # Refresh canvas after updating the plot
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2